In [16]:
# Intraplant PPV
# importing modules
import shutil  # for file manipulation
import os
import win32com.client as win32
from ok_sap_script import *
import time
from datetime import date, timedelta, datetime, timezone
import pyautogui

class Control():
    def __init__(self):
        # main variables
        self.curr_user = os.getlogin()
        self.date_stamp = datetime.today().strftime("%d-%m-%Y")
        self.receiver = "oleksandr.komarov@zoetis.com"
        
        # SAP variables 
        self.environment = "ECC Production"
        self.t_code_sap = "ZP2M_TRACKSTDP"
        self.variant_sap = ["COST_METHOD_2", "COST_METHOD_1"] # we run it twice: international and local
        self.screenshots = [f'scr_1_for_{self.date_stamp}.png',
                            f'scr_2_for_{self.date_stamp}.png',
                            f'scr_3_for_{self.date_stamp}.png',
                            f'scr_4_for_{self.date_stamp}.png',
                            f'scr_final_for_{self.date_stamp}.png']

        # Paths to files
            # model
        self.model_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Costing_methodology/Costing_methodology_model.xlsx"
        self.new_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Costing_methodology/Output/Costing_methodology_for_{self.date_stamp}.xlsx"
            # screenshot
        self.screenshot_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Costing_methodology/Screenshots/"

    def __call__(self, *args, **kwargs):
        self.run_logic_sap(self.t_code_sap)
        self.run_logic_excel()

    def sap_decorator(sap_function):
        '''Closing SAP sessions before and after running a script'''
        def sap_wrapper(self, t_code_sap, *args, **kwargs):
            sap_close() # ensure there are no open session
            open_sap = sap_open() # open new session
            time.sleep(5) # make sure SAP opens up
            if open_sap == True:
                self.session = sap_logon(environment=self.environment, client=1)
                sap_code(tcode=t_code_sap, session=self.session)
                sap_function(self) # this is the main fuction to be decorated
            else:
                print("SAP did not open")
            sap_close() # close SAP
        return sap_wrapper

    def excel_decorator(excel_function):
        '''Opening Excel, applying some standard parameters for runtime optimisation, etc.'''
        def excel_wrapper(self, *args, **kwargs):
            try:
                # default Excel runtime optimisation
                self.excel = win32.Dispatch("Excel.Application")
                self.excel.AskToUpdateLinks = False
                self.excel.DisplayAlerts = False
                self.excel.Visible = True
                self.excel.ScreenUpdating = False
                excel_function(self) # our main Excel function to run
                #ToDo
            except Exception as e:
                # return to a normal Excel, then close it
                self.excel.ScreenUpdating = True
                self.excel.Application.Calculation = -4105  # to set xlCalculationAutomatic
                close_excel()
                print(f"something went wrong, namely\n{e}")
        return excel_wrapper
    
    @excel_decorator
    def run_logic_excel(self): # extract historical cost from the shared drive
        # copying main model file
        try:
            shutil.copy(self.model_file_path, self.new_file_path)
        except PermissionError as e:
            print("The file may be opened")
            os.system("taskkill /f /im  excel.exe")
    # open created Excel file and perform the following manipulations
            # open new file and disable calculations
        self.new_file = self.excel.Workbooks.Open(self.new_file_path)
        self.excel.Application.Calculation = (-4135)  # to set xlCalculationManual # Workbook needs to be opened
            # select worksheet and populate screenshots
        self.sheets = ["Costing methodology", "Costing methodology LRDs"]
        try:
            for sheet in self.sheets:
                self.new_file_ws = self.new_file.Sheets(sheet)
                # add screenshots for local
                self.pictures = self.new_file_ws.Pictures()
                for pic in self.pictures:
                    pic.Delete()
                self.left, self.top, self.width, self.height = 0, 10, 950, 640
                for screenshot in self.screenshots:
                    self.picture_filename = screenshot
                    if sheet == self.sheets[0]:
                        self.picture_path = fr'C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\Costing_methodology\\Screenshots\\loc\\{self.picture_filename}'
                    else:
                        self.picture_path = fr'C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\Costing_methodology\\Screenshots\\int\\{self.picture_filename}'    
                # insert a new screenshot with given parameters
                    self.picture = self.new_file_ws.Shapes.AddPicture(self.picture_path, LinkToFile=False, SaveWithDocument=True, Left=self.left, Top=self.top, Width=self.width, Height=self.height)
                    self.left += self.width
                # insert two extra screenshots
                self.new_file_ws = self.new_file.Sheets("Costing methodology LRDs")
                self.picture_path = fr'C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\Costing_methodology\\Screenshots\\int\\Extra_screen1_for_{self.date_stamp}.png'
                self.left, self.top, self.width, self.height = 0, 640, 950, 640
                self.picture = self.new_file_ws.Shapes.AddPicture(self.picture_path, LinkToFile=False, SaveWithDocument=True, Left=self.left, Top=self.top, Width=self.width, Height=self.height)
        
                self.picture_path = fr'C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\Costing_methodology\\Screenshots\\int\\Extra_screen2_for_{self.date_stamp}.png'
                self.left, self.top, self.width, self.height = 950, 640, 950, 640
                self.picture = self.new_file_ws.Shapes.AddPicture(self.picture_path, LinkToFile=False, SaveWithDocument=True, Left=self.left, Top=self.top, Width=self.width, Height=self.height)
        except Exception as e:
            print(f"No screenshot or {e}")
        finally:
            # back to default
            self.excel.ScreenUpdating = True
            self.excel.Application.Calculation = -4105  # to set xlCalculationAutomatic
            self.new_file.Save()
            self.new_file.Close()
            close_excel()
            self.send_email() # send generated file by email

    def send_email(self):
        # send by email
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = self.receiver
            mail.Subject = f"Automatic email for Costing methodology control for {self.date_stamp}"
            mail.Body = f"Please consult the attachment"
            mail.Attachments.Add(f"{self.new_file_path}")
            #mail.Display(True)
            mail.Send()

    @sap_decorator
    def run_logic_sap(self):
        # self.session is defined inside sap_decorator
        for variant in self.variant_sap:
            try:
                sap_variant(session=self.session, var_to_use=variant)
            except: 
                sap_variant(session=self.session, var_to_use=variant, version=2) # in our case, V2 is the most probable scenario
            # define folder
            if variant == self.variant_sap[0]:
                self.folder = "loc"
            else: self.folder = "int"
            # take screenshots
                #1
            self.session.findById("wnd[0]/usr/btn%_SP$00002_%_APP_%-VALU_PUSH").press()
            self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpINTL").select()
            self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpINTL/ssubSCREEN_HEADER:SAPLALDB:3020/tblSAPLALDBINTERVAL/ctxtRSCSEL_255-ILOW_I[1,3]").setFocus()
            # screenshot
            try:
                screenshot_first_page = pyautogui.screenshot()
                screenshot_first_page.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Costing_methodology/Screenshots/{self.folder}/{self.screenshots[0]}')
            except Exception as e: print(e)
            self.session.findById("wnd[1]").close() 
                #2
            self.session.findById("wnd[0]/usr/btn%_SP$00006_%_APP_%-VALU_PUSH").press()
            if variant == self.variant_sap[0]:
                self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,4]").setFocus()
            else:
                self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOSV").select()
                self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOSV/ssubSCREEN_HEADER:SAPLALDB:3030/tblSAPLALDBSINGLE_E/ctxtRSCSEL_255-SLOW_E[1,1]").setFocus()
            try:
                screenshot_first_page = pyautogui.screenshot()
                screenshot_first_page.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Costing_methodology/Screenshots/{self.folder}/{self.screenshots[1]}')
            except Exception as e: print(e)
            if variant == self.variant_sap[1]:
                self.session.findById("wnd[1]").sendVKey(82)
                time.sleep(1)
                try:
                    screenshot_first_page = pyautogui.screenshot()
                    screenshot_first_page.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Costing_methodology/Screenshots/{self.folder}/Extra_screen1_for_{self.date_stamp}.png')
                except Exception as e: print(e)

            self.session.findById("wnd[1]").close()
            if variant == self.variant_sap[1]: self.session.findById("wnd[0]/usr/ctxtSP$00007-LOW").text = ""
                #3
            self.session.findById("wnd[0]/usr/btn%_SP$00007_%_APP_%-VALU_PUSH").press()
            self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOSV").select()
            self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOSV/ssubSCREEN_HEADER:SAPLALDB:3030/tblSAPLALDBSINGLE_E/ctxtRSCSEL_255-SLOW_E[1,3]").setFocus()
            try:
                screenshot_first_page = pyautogui.screenshot()
                screenshot_first_page.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Costing_methodology/Screenshots/{self.folder}/{self.screenshots[2]}')
            except Exception as e: print(e)
            self.session.findById("wnd[1]").close()
                #4
            self.session.findById("wnd[0]/usr/btn%_SP$00010_%_APP_%-VALU_PUSH").press()
            if variant == self.variant_sap[0]:
                self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOINT").select()
                self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOINT/ssubSCREEN_HEADER:SAPLALDB:3040/tblSAPLALDBINTERVAL_E/ctxtRSCSEL_255-ILOW_E[1,1]").setFocus()
            else:
                self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOSV").select()
                self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOSV/ssubSCREEN_HEADER:SAPLALDB:3030/tblSAPLALDBSINGLE_E/ctxtRSCSEL_255-SLOW_E[1,1]").setFocus()    
            try:
                screenshot_first_page = pyautogui.screenshot()
                screenshot_first_page.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Costing_methodology/Screenshots/{self.folder}/{self.screenshots[3]}')
            except Exception as e: print(e)
            if variant == self.variant_sap[1]: # taking extra screenshot for international tab
                self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOINT").select()
                self.session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOINT/ssubSCREEN_HEADER:SAPLALDB:3040/tblSAPLALDBINTERVAL_E/ctxtRSCSEL_255-ILOW_E[1,1]").setFocus()
                try:
                    screenshot_first_page = pyautogui.screenshot()
                    screenshot_first_page.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Costing_methodology/Screenshots/{self.folder}/Extra_screen2_for_{self.date_stamp}.png')
                except Exception as e: print(e)
            self.session.findById("wnd[1]").close()
            sap_run(session=self.session)
            self.result = self.session.findById("wnd[0]/sbar").Text
            # take final screenshot
            try:
                screenshot_first_page = pyautogui.screenshot()
                screenshot_first_page.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Costing_methodology/Screenshots/{self.folder}/{self.screenshots[4]}')
            except Exception as e: print(e)
    
control = Control()
control()
#control.run_logic_sap_fagll03(control.t_code_sap)
#control.run_logic_sap_faglb03(control.t_code_sap_faglb03)

In [ ]:
import numpy as np
import pandas as pd

#--GIT APxx-xx tab:
# looks like a very nasty .txt file, which requires specific approach
df_sap = pd.read_csv("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/GIT_AP02.txt", 
                     on_bad_lines='skip', sep="\t", encoding="ANSI",skiprows=13, skipinitialspace = True)
df_sap = df_sap.loc[:, ~df_sap.columns.str.contains('^Unnamed')] # drop all unnamed columns -> ~ stands for bool
df_sap = df_sap[df_sap['Plant'] != "Plant"] # drop all rows that have "Plant" in their names (those are repetitions of headers)
df_sap.dropna(subset=['Plant'], inplace=True)
# convert respective columns to numeric
df_sap['Quantity'] = df_sap['Quantity'].str.replace(',', '').astype(float)
df_sap['Amount in LC'] = df_sap['Amount in LC'].str.replace(',', '').astype(float)
df_sap['Net Order Value in PO Curr.'] = df_sap['Net Order Value in PO Curr.'].str.replace(',', '').astype(float)
df_sap['PO Quantity'] = df_sap['PO Quantity'].str.replace(',', '').astype(float)
# export to excel
#df_sap.to_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/Pandas_GIT_AP02.xlsx", index=False)

#--Data Reworked APxx-xx tab:
df_sap_le = df_sap.copy() # creating a copy of the file with only 4 LEs 
df_sap_le = df_sap_le[(df_sap_le['Company Code'] == "2941") | (df_sap_le['Company Code'] == "2942") | 
                      (df_sap_le['Company Code'] == "2946") | (df_sap_le['Company Code'] == "2951")]
# reset index:
df_sap_le = df_sap_le.reset_index(drop=True)
# add index column:
df_sap_le['INDEX'] = df_sap_le.index + 2
# adding columns:
# concatenate:
df_sap_le.insert(loc=0, column='Concatenate', value=df_sap_le['Material Number'] + "-" + df_sap_le['Plant'])
# other:
df_sap_le['BUoM historic'] = "=VLOOKUP(A" + df_sap_le['INDEX'].astype(str) + ",historicalcostsv2,7,FALSE)"
df_sap_le['Std price per BUoM'] = "=VLOOKUP(A" + df_sap_le['INDEX'].astype(str) + ",historicalcostsv2,9,FALSE)"
df_sap_le['Value at historical cost'] = "=IF(AJ" + df_sap_le['INDEX'].astype(str) + "=AC" + df_sap_le['INDEX'].astype(str) + ",0,D" + df_sap_le['INDEX'].astype(str) + "*AO" + + df_sap_le['INDEX'].astype(str) + ")"
df_sap_le['Test UOM'] = "=AN" + df_sap_le['INDEX'].astype(str) + "=E" + df_sap_le['INDEX'].astype(str)
df_sap_le['Diff $'] = "=IF(AJ" + df_sap_le['INDEX'].astype(str) + "=AC" + df_sap_le['INDEX'].astype(str) + ",0,AP" + df_sap_le['INDEX'].astype(str) + "-F" + df_sap_le['INDEX'].astype(str) + ")"
print(df_sap_le)
# drop index colums:
df_sap_le = df_sap_le.drop('INDEX', axis=1)
# export to excel:
#df_sap_le.to_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/Pandas_GIT_Reworked.xlsx", index=False)


#print(df_sap_le)


# second part (work with mm60 extract)

In [ ]:
import numpy as np
import pandas as pd

df_mm60 = pd.read_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/Historical cost AP02 LE2941.xlsm", 
                        sheet_name="MM60 Report", converters={'Price':float})

pd.options.display.float_format = '{:20,.2f}'.format # handling scientific notation

# add columns:
#df_mm60['INDEX'] = df_mm60.index + 2
df_mm60.insert(loc=0, column='Concatenate', value=df_mm60['Material'] + "-" + df_mm60['Plant'])
df_mm60['Std price per unit'] = df_mm60['Price'] / df_mm60['Price unit']
print(df_mm60)
df_mm60.to_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/pandas_mm60_rpa.xlsx", index=False)

# if historical cost file is received from Roel

In [84]:
import numpy as np
import pandas as pd

df_roel = pd.read_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/historical costs AP 2 2023 - LE 2941.xlsx",
                        converters={'BusA': str})
# remove all unwanted columns:
df_roel = df_roel[['Plnt', 'Material', 'BusA', 'Standard price', 'Crcy', 'BUn', 'per']]
# add useful columns:
df_roel.insert(loc=0, column='Concatenate', value=df_roel['Material'] + "-" + df_roel['Plnt'])
df_roel['Std price per unit'] = df_roel['Standard price'] / df_roel['per']
# drop last row:
df_roel.drop(df_roel.tail(1).index,inplace=True) # drop last (n) rows

print(df_roel)

          Concatenate  Plnt  Material  BusA       Standard price  Crcy  BUn  \
0       10009694-AT00  AT00  10009694  9334                 6.33   NaN  SYR   
1       10020218-AT00  AT00  10020218  9334            19,562.83   NaN   PC   
2       10020219-AT00  AT00  10020219  9334               861.93   NaN   PC   
3       10020220-AT00  AT00  10020220  9334            17,135.68   NaN   PC   
4       10020221-AT00  AT00  10020221  9334             5,327.76   NaN   PC   
...               ...   ...       ...   ...                  ...   ...  ...   
107048  25001338-IE11  IE11  25001338  7246                 5.67   NaN   SU   
107049  25001339-IE11  IE11  25001339  7246                12.05   NaN   SU   
107050  25001340-IE11  IE11  25001340  7246                35.70   NaN   SU   
107051  25001341-IE11  IE11  25001341  7246                54.44   NaN   SU   
107052  25001342-IE11  IE11  25001342  7246                33.00   NaN   SU   

                        per   Std price per unit  
